<a href="https://colab.research.google.com/github/Pushkar1853/C-practise/blob/master/prediction_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 9.9 MB/s 
     |████████████████████████████████| 7.6 MB 63.7 MB/s 
     |████████████████████████████████| 182 kB 66.9 MB/s 


In [ ]:
import argparse
import collections
import json
# import numpy as np
import os
import re
import string
import sys
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

In [ ]:
model_name = "deepset/roberta-base-squad2"

In [ ]:
# nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
# QA_input = {
#     'question': 'From which countries did the Norse originate?',
#     'context': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.'
# }
# res = nlp(QA_input)

In [ ]:
# res

In [ ]:
test_squad = pd.read_csv('/content/drive/MyDrive/Inter IIT High Prep/squad_validation_data.csv')

In [ ]:
test_squad['context'][2]

'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.'

In [ ]:

test_squad['context'].unique()

array(['The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.',
       'The Norman dynasty had a major political, cultural and military impact on medieval Europe and even the Near East. The Normans were famed for their martial spirit and eventually for their Christian piety, becoming exponents of the Catholic 

In [ ]:
test_squad

,question,context,answer_start,text
0,In what country is Normandy located?,The Normans (Norman: Nourmands; French: Norman...,159,France
1,When were the Normans in Normandy?,The Normans (Norman: Nourmands; French: Norman...,94,10th and 11th centuries
2,From which countries did the Norse originate?,The Normans (Norman: Nourmands; French: Norman...,256,"Denmark, Iceland and Norway"
3,Who was the Norse leader?,The Normans (Norman: Nourmands; French: Norman...,308,Rollo
4,What century did the Normans first gain their ...,The Normans (Norman: Nourmands; French: Norman...,671,10th century
...,...,...,...,...
11868,What is the seldom used force unit equal to on...,"The pound-force has a metric counterpart, less...",665,sthène
11869,What does not have a metric counterpart?,"The pound-force has a metric counterpart, less...",[],[]
11870,What is the force exerted by standard gravity ...,"The pound-force has a metric counterpart, less...",[],[]
11871,What force leads to a commonly used unit of mass?,"The pound-force has a metric counterpart, less...",[],[]


In [ ]:
def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()
  return white_space_fix(remove_articles(remove_punc(lower(s))))

In [ ]:
def get_tokens(s):
  if not s: return []
  return normalize_answer(s).split()

In [ ]:
def compute_f1(a_gold, a_pred):
  gold_toks = get_tokens(a_gold)
  pred_toks = get_tokens(a_pred)
  common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
  num_same = sum(common.values())
  if len(gold_toks) == 0 or len(pred_toks) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    return int(gold_toks == pred_toks)
  if num_same == 0:
    return 0
  precision = 1.0 * num_same / len(pred_toks)
  recall = 1.0 * num_same / len(gold_toks)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1


In [ ]:
def prediction(data, model_name):
  text = []
  start_idx = []
  model = pipeline('question-answering', model=model_name, tokenizer=model_name)
  for i in tqdm(range(test_squad.shape[0])):
    question = test_squad['question'][i]
    context = test_squad['context'][i]
    input = {
      'question': question,
      'context': context
  }
    res = model(input)
    if(res['score']>0.50):
      txt = res['answer']
      st_idx = res['start']
    else:
      txt = ' '
      st_idx = ' '
    text.append(txt)
    start_idx.append(st_idx)

  return text, start_idx


In [ ]:
text, start_idx = prediction(test_squad, model_name)

  0%|          | 0/11873 [00:00<?, ?it/s]

In [ ]:
text[0: 10]

['France',
 '10th and 11th centuries',
 ' ',
 ' ',
 '10th',
 ' ',
 ' ',
 ' ',
 ' ',
 'William the Conqueror']

In [ ]:
test_squad['text'][0: 10]

0                         France
1        10th and 11th centuries
2    Denmark, Iceland and Norway
3                          Rollo
4                   10th century
5                             []
6                             []
7                             []
8                             []
9          William the Conqueror
Name: text, dtype: object